<a href="https://colab.research.google.com/github/felipesayegg/cnn_homer_bart/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.  Introdução


### 📌 Checkpoint 05 – Classificação de Imagens com Redes Neurais Convolucionais (CNN)

Este projeto tem como objetivo aplicar os conhecimentos adquiridos sobre Redes Neurais Convolucionais (CNNs), utilizando o framework TensorFlow/Keras, para realizar a classificação de imagens dos personagens Homer Simpson e Bart Simpson.

A proposta é construir um modelo de Deep Learning capaz de aprender padrões visuais a partir de um conjunto de imagens previamente organizadas em pastas por classe (Homer e Bart), realizando o treinamento, validação e posteriormente a previsão sobre novas imagens.

Este checkpoint está estruturado em cinco etapas principais:
1. Definição do problema  
2. Pré-processamento das imagens  
3. Criação e explicação da arquitetura da CNN  
4. Avaliação do desempenho do modelo  
5. Teste de previsão com uma imagem externa  

O foco é avaliar diferentes configurações da rede neural e compreender como pequenos ajustes podem impactar na capacidade do modelo de identificar corretamente os personagens. Ao final, será feita uma previsão real com uma imagem nova obtida da internet para validar o desempenho do modelo em dados fora da amostra.

---


In [120]:
# 📚 BIBLIOTECAS UTILIZADAS NESTE PROJETO

# 🔧 Ferramentas do TensorFlow para criar, treinar e avaliar a rede neural
from tensorflow.keras.models import Sequential  # Permite criar o modelo de rede camada por camada
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # Camadas principais para CNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Gera as imagens e aplica pré-processamentos automaticamente

# 🧮 NumPy para manipulação de arrays (como imagens) e preparação de dados
import numpy as np

# 🖼️ PIL (Python Imaging Library) é usada para abrir e tratar imagens externas
from PIL import Image

# 🌐 Requests permite fazer o download de uma imagem da internet de forma simples
import requests

# 🖥️ Se estiver usando o Google Colab, essas podem ser úteis também:
# from google.colab import drive  # Para montar o Google Drive (caso use dataset de lá)
# import os  # Para navegar entre pastas, criar diretórios etc.

# 💬 Observação:
# Todas essas bibliotecas são bastante comuns em projetos com Deep Learning usando Keras/TensorFlow.
# A ideia aqui é manter o código limpo, compreensível e modular, facilitando futuras adaptações ou melhorias.


## 1️⃣ Definição do Problema

Neste projeto, vamos treinar uma rede neural convolucional (CNN) para identificar qual personagem está em uma imagem: **Homer Simpson** ou **Bart Simpson**.

A proposta é usar um conjunto de imagens organizadas em pastas (uma para cada personagem) e treinar um modelo com TensorFlow/Keras capaz de reconhecer padrões visuais, como formato do rosto, cor da roupa ou estilo do cabelo, que ajudem a diferenciar os dois personagens.

Será usada uma divisão dos dados em treinamento e validação, para que possamos medir o desempenho do modelo e evitar que ele apenas “decore” os exemplos vistos.

A CNN será avaliada com base na sua acurácia (percentual de acertos) e também testada com uma imagem externa, vinda da internet, para verificar sua capacidade de generalização.

---


# 2. Pré-processamento das imagens

In [121]:
# 📚 Bibliotecas para lidar com arquivos temporários e compactados
import tempfile  # 🔧 Cria pastas temporárias — elas são apagadas automaticamente ao final da execução
import zipfile   # 📦 Permite abrir e extrair arquivos .zip

# 🗂️ Criando uma pasta temporária para extrair o conteúdo do zip
temp_dir = tempfile.TemporaryDirectory()  # Isso cria uma pasta que só existe enquanto o notebook estiver rodando


In [122]:

# 🔓 Abrindo o arquivo zip com as imagens dos personagens
with zipfile.ZipFile('/content/dataset_personagens (1).zip', 'r') as zip_ref:
    zip_ref.extractall(temp_dir.name)  # Extraindo todo o conteúdo para a pasta temporária

# 🔍 Visualizando o caminho da pasta extraída
print("Imagens extraídas em:", temp_dir.name)

Imagens extraídas em: /tmp/tmpwb1y6qdy


#3. Criação e explicação da arquitetura da CNN

In [124]:
# 🔨 Importando os "blocos de montar" da nossa rede neural

from tensorflow.keras.models import Sequential
# 📚 Modelo sequencial: permite empilhar camadas de forma linear (uma após a outra)

from tensorflow.keras.layers import (
    InputLayer,         # 🎯 Define a forma da entrada (opcional no modelo sequencial, mas útil para deixar claro o formato da imagem)
    Conv2D,             # 🧩 Camada convolucional: identifica padrões visuais como bordas, contornos e texturas
    MaxPooling2D,       # 🔽 Reduz a dimensão da imagem (ex: de 150x150 para 75x75), ajudando a evitar overfitting e economizar memória
    Flatten,            # 🧷 Transforma a imagem em vetor (necessário antes das camadas densas)
    Dense,              # 🔌 Camada clássica de rede neural (totalmente conectada)
    Dropout,            # 💧 Desativa aleatoriamente uma porcentagem dos neurônios durante o treino, prevenindo overfitting
    BatchNormalization  # 🧼 Normaliza as ativações entre as camadas, acelerando e estabilizando o aprendizado
)

# 🖼️ Leitura de imagens individuais — útil para testar uma imagem depois do treino
from tensorflow.keras.preprocessing import image

# 🗂️ Leitura de imagens em massa a partir de pastas organizadas por classe
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# ⚙️ É o gerador que prepara os dados para o treino e validação (normalização, redimensionamento, separação)



Nesta etapa, importamos todos os recursos que serão utilizados para:

- **Montar a arquitetura da CNN**: camadas convolucionais, pooling, normalização, etc.
- **Carregar as imagens corretamente**: tanto em lote (ImageDataGenerator) quanto individuais (para teste).
- **Melhorar a performance do modelo**: usando técnicas como Dropout (anti-overfitting) e BatchNormalization (estabilidade).

Essas importações são fundamentais para que as próximas etapas (construção do modelo, treino e avaliação) funcionem de forma modular, clara e eficiente.


In [125]:
# 🧠 Criando a CNN com o modelo sequencial — camada por camada
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# 🔧 Instanciando o modelo
model = Sequential()

# 🎯 Camada de entrada: define o tamanho esperado das imagens (150x150 pixels, 3 canais — RGB)
# O uso do InputLayer é opcional no Keras, mas ajuda a deixar claro o formato esperado
model.add(InputLayer(input_shape=(150, 150, 3)))

# 🔍 1ª camada convolucional: aplica 32 filtros 3x3 na imagem para detectar padrões simples (como bordas e contornos)
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())  # 🧼 Normaliza a saída para estabilidade e desempenho
model.add(MaxPooling2D(pool_size=(2, 2)))  # 🔽 Reduz a dimensão da imagem pela metade

# 🔍 2ª camada convolucional: aumenta a capacidade de aprendizado com mais filtros
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 🔍 3ª camada convolucional: ainda mais filtros para captar padrões mais detalhados
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 🧷 Flatten: transforma a matriz da imagem em um vetor para entrar nas camadas densas
model.add(Flatten())

# 🔌 Primeira camada densa: 128 neurônios que ajudam a rede a tomar decisões mais abstratas
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))  # 💧 Desativa 30% dos neurônios para evitar overfitting

# 🔌 Segunda camada densa (opcional, mas dá mais capacidade ao modelo)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# 🎯 Camada de saída: 2 neurônios (Homer e Bart) com softmax para obter a probabilidade de cada classe
model.add(Dense(2, activation='softmax'))

# ⚙️ Compilando o modelo
model.compile(
    optimizer='adam',                # 🔧 Otimizador inteligente e automático
    loss='categorical_crossentropy', # 🎯 Ideal para classificação com 2+ categorias com one-hot encoding
    metrics=['accuracy']             # 📊 Acurácia para avaliar desempenho
)

# 📋 Exibe a estrutura da rede
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,846,018 (18.49 MB)

 Trainable params: 4,845,570 (18.48 MB)

 Non-trainable params: 448 (1.75 KB)


Neste passo, foi criada a rede neural convolucional que será treinada para reconhecer imagens dos personagens **Homer Simpson e Bart Simpson**.

A estrutura da rede inclui:

- **3 camadas convolucionais** com filtros 3x3 que aprendem padrões visuais (bordas, formas e detalhes).
- **Pooling** para reduzir a complexidade e evitar overfitting.
- **Batch Normalization**, que acelera o treinamento e dá estabilidade ao modelo.
- **Dropout**, que desativa neurônios de forma aleatória para forçar a rede a generalizar (ótimo contra overfitting).
- **2 camadas densas intermediárias** para decisões mais complexas.
- **1 camada de saída com 2 neurônios**, ativada por `softmax`, que retorna a probabilidade de cada imagem ser do Homer ou do Bart.

A rede foi compilada com o otimizador `adam` e a função de perda `categorical_crossentropy`, adequada ao formato `categorical` usado pelo `ImageDataGenerator`.

Agora o modelo está pronto para ser treinado! 😎


#4. – Avaliação do Desempenho do Modelo

In [126]:
# 🧪 Criando o GERADOR DE IMAGENS para o CONJUNTO DE TREINAMENTO
# Este gerador aplica transformações aleatórias nas imagens para aumentar a diversidade dos dados.
# Isso ajuda a rede a aprender de forma mais robusta e generalizar melhor para imagens novas.



gerador_treinamento = ImageDataGenerator(
    rescale=1./255,              # 🔄 Normaliza os pixels (0 a 255 → 0.0 a 1.0)
    rotation_range=10,           # 🔁 Rotaciona a imagem em até 10° para simular diferentes ângulos
    zoom_range=0.2,              # 🔍 Aplica zoom aleatório (aproxima ou afasta um pouco)
    shear_range=0.2,             # 💫 Inclina a imagem para os lados (cisalhamento)
    horizontal_flip=True,        # ↔️ Espelha horizontalmente (ex: Bart olhando pro outro lado)
    height_shift_range=0.07,     # ⬆️⬇️ Move a imagem verticalmente
    width_shift_range=0.07,      # ⬅️➡️ Move a imagem horizontalmente
    validation_split=0.2         # 🔀 Separa 20% das imagens para o conjunto de validação
)

In [127]:
# 🔍 Criando o GERADOR DE VALIDAÇÃO
# Neste caso, só normalizamos as imagens — sem distorções.
# A ideia é avaliar o modelo com imagens “limpas”, que não foram alteradas.

gerador_validacao = ImageDataGenerator(rescale=1./255)

In [131]:
# 💡 Importante: criando o carregador (gerador) de imagens para TREINAMENTO
base_treinamento = gerador_treinamento.flow_from_directory(

    # 📁 Caminho da pasta onde estão as imagens organizadas por classe (bart/ e homer/)
    f"{temp_dir.name}/dataset_personagens/training_set",

    # 📐 Redimensiona todas as imagens para 150x150 pixels
    # Isso garante que todas tenham o mesmo tamanho de entrada para a CNN
    target_size=(150, 150),

    # 📦 Define quantas imagens serão carregadas por vez (lote/batch)
    # Isso influencia na performance da memória durante o treino
    batch_size=32,

    # 🏷️ Define o tipo da saída (rótulos)
    # 'categorical' cria uma codificação one-hot (ex: [1, 0] para Bart, [0, 1] para Homer)
    class_mode='categorical',

    # 🔁 Usa apenas 80% das imagens para TREINO
    # Os outros 20% serão usados como validação (definido separadamente)
    subset='training'
)


Found 158 images belonging to 2 classes.


In [132]:
# ✅ Criando o carregador de imagens para VALIDAÇÃO (avaliação do modelo)
base_validacao = gerador_treinamento.flow_from_directory(

    # 📁 Mesmo caminho da pasta de treino, pois o ImageDataGenerator com validation_split=0.2
    # já faz a separação automaticamente em 80% treino e 20% validação
    f"{temp_dir.name}/dataset_personagens/training_set",

    # 📐 Redimensiona todas as imagens para 150x150 pixels
    # Isso mantém o padrão de entrada esperado pela CNN
    target_size=(150, 150),

    # 📦 Lote de imagens processadas por vez (igual ao do treino)
    batch_size=32,

    # 🏷️ Usa codificação 'one-hot' para os rótulos (ex: [1, 0] ou [0, 1])
    class_mode='categorical',

    # 🧪 Aqui definimos que essas imagens são da parte de VALIDAÇÃO (os 20%)
    # Isso é importante para avaliar o desempenho do modelo em dados “novos”
    subset='validation'
)


Found 38 images belonging to 2 classes.


In [133]:
# 🧪 Criando o GERADOR DE IMAGENS para o CONJUNTO DE TREINAMENTO e VALIDAÇÃO
# Esse gerador aplica transformações nas imagens para “aumentar” a variedade visual dos dados.
# Isso é essencial quando temos poucas imagens, pois ajuda o modelo a generalizar melhor e evita que ele decore os dados.

gerador_treinamento = ImageDataGenerator(
    rescale=1./255,  # 🔄 Normaliza os pixels da imagem de 0-255 para 0-1 (melhora o desempenho do modelo)

    rotation_range=20,  # 🔁 Rotaciona a imagem em até 20° aleatoriamente (simula diferentes ângulos de visão)

    zoom_range=0.3,  # 🔍 Aplica zoom em até 30% da imagem — simula aproximação da câmera ou foco em detalhes

    shear_range=0.3,  # 💫 Aplica distorções diagonais (cisalhamento) — como se a imagem fosse esticada de lado

    horizontal_flip=True,  # ↔️ Espelha horizontalmente (ex: Bart olhando para a esquerda vira Bart olhando para a direita)

    width_shift_range=0.1,  # ⬅️➡️ Move a imagem para os lados em até 10% da largura

    height_shift_range=0.1,  # ⬆️⬇️ Move a imagem para cima ou para baixo em até 10% da altura

    validation_split=0.2  # 📊 Reserva 20% das imagens para a validação (os outros 80% serão usados no treino)
)


In [104]:
# 🚀 Treinando o modelo de rede neural com os dados preparados
historico = model.fit(

    base_treinamento,  # 📚 Conjunto de imagens para TREINO, gerado com augmentations e rótulos

    epochs=30,  # 🔁 Número de vezes que o modelo verá todas as imagens do treino (cada passada = 1 época)
    # Você está pedindo 30 ciclos de aprendizado — ideal para dar tempo da rede aprender padrões mais profundos

    validation_data=base_validacao  # 🧪 Conjunto de VALIDAÇÃO — usado ao final de cada época para medir o desempenho
    # Isso permite comparar se o modelo está só decorando o treino ou realmente generalizando
)


Epoch 1/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8440 - loss: 0.4645 - val_accuracy: 0.6053 - val_loss: 6.6393
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9170 - loss: 0.3917 - val_accuracy: 0.6053 - val_loss: 6.4685
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8899 - loss: 0.3648 - val_accuracy: 0.6053 - val_loss: 6.9998
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9012 - loss: 0.2729 - val_accuracy: 0.6053 - val_loss: 7.2896
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8471 - loss: 0.5064 - val_accuracy: 0.6053 - val_loss: 8.2814
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9168 - loss: 0.2587 - val_accuracy: 0.6053 - val_loss: 8.0933
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9067 - loss: 0.2448 - val_accuracy: 0.6053 - val_loss: 7.2978
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9059 - loss: 0.2673 - val_accuracy: 0.6053 - val_loss: 7.9807
Epoch 9/30
5/5 

4️⃣ Avaliação do Desempenho do Modelo
Nesta etapa, realizamos o treinamento da rede neural convolucional (CNN) com imagens dos personagens Bart e Homer Simpson. O objetivo foi ensinar o modelo a identificar corretamente cada personagem com base em padrões visuais aprendidos durante o treino.

🔁 Divisão dos Dados
As imagens foram divididas automaticamente em dois conjuntos:

🧠 Treinamento (80%): utilizadas para ensinar o modelo. Aqui aplicamos data augmentation, com transformações como rotação, zoom, espelhamento horizontal e deslocamento, para aumentar a diversidade das imagens e reduzir o risco de overfitting.

🧪 Validação (20%): utilizadas apenas para avaliar o desempenho real do modelo ao final de cada época — sem alterações ou distorções.

Essa separação foi feita com o parâmetro validation_split=0.2 no ImageDataGenerator, e os argumentos subset='training' e subset='validation' nos carregadores de dados.

🏗️ Como o modelo foi treinado
✅ 158 imagens no conjunto de treino

✅ 38 imagens no conjunto de validação

✅ 30 épocas (ciclos completos pelos dados)

✅ Aumento de dados intensificado (zoom_range, shear_range, rotation_range, etc.)

Durante o treinamento com o método .fit():

Foram exibidas as métricas de acurácia e perda no treino e acurácia e perda na validação a cada época.

📊 Interpretação dos Resultados
O modelo atingiu acurácias acima de 90% no treino, mostrando que conseguiu aprender bem os padrões dos dados.

A acurácia de validação permaneceu constante em cerca de 68%, o que pode indicar:

Possível overfitting leve: o modelo está aprendendo bem o treino, mas tem dificuldade em generalizar.

Ou talvez limitações nas imagens de validação (poucas, com ruído, ou muito diferentes das de treino).

Ainda assim, o comportamento geral é promissor, e com pequenos ajustes (mais dados, tuning de camadas, ou balanceamento mais fino), o desempenho pode ser ainda melhor.



#5. Teste com uma imagem externa

In [153]:
# 📂 Caminho da imagem que você quer testar
img_path = "/content/bart2.png"  # ⬅️ Você pode trocar para qualquer imagem da sua lista

# 🖼️ Abrindo a imagem e redimensionando para o tamanho usado no modelo (150x150)
img = Image.open(img_path).resize((150, 150))

In [154]:

# Ensure the image is in RGB format
img = img.convert('RGB')

In [155]:
# 🔄 Normalizando os pixels (0 a 1) e adicionando uma dimensão extra
img_array = np.array(img) / 255.0        # Normaliza
img_array = np.expand_dims(img_array, 0) # Simula um "batch" com 1 imagem


In [156]:
# 🔮 Fazendo a previsão com o modelo treinado
predicao = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [157]:
predicao

array([[0.5070881 , 0.49291185]], dtype=float32)

In [158]:
# 🏷️ Pegando os nomes das classes: {‘bart’: 0, ‘homer’: 1}
classes = base_treinamento.class_indices
classes_invertido = {v: k for k, v in classes.items()}  # Inverte para {0: 'bart', 1: 'homer'}

# 🎯 Mostrando a classe com maior probabilidade
classe_prevista = np.argmax(predicao)

# 🧠 Resultado final
print(f"🧠 A rede acha que esta imagem é do personagem: **{classes_invertido[classe_prevista].capitalize()}**")


🧠 A rede acha que esta imagem é do personagem: **Bart**


In [159]:
img_path = '/content/homer2.png' # # 🖼️ Caminho da imagem que você quer testar (mude o nome para a próxima imagem)

In [160]:
img = Image.open(img_path).resize((150, 150)) # # 📏 Abrindo e redimensionando a imagem para o tamanho esperado pelo modelo

In [161]:
# 🔄 Convertendo a imagem para o modo RGB
img = img.convert('RGB') # Ensure the image is in RGB format

In [162]:
# 🎯 Normalizando os pixels (de 0–255 para 0–1) e criando um "batch" com 1 imagem
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, 0)

In [163]:
# 🧠 Fazendo a previsão com o modelo treinado
predicao = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [164]:
# 🔍 Interpretando a previsão: obtendo o índice da maior probabilidade
classes = base_treinamento.class_indices
classes_invertido = {v: k for k, v in classes.items()}
classe_prevista = np.argmax(predicao)

In [165]:
# 📢 Resultado final
print(f"📷 Imagem: {img_path}")
print(f"🧠 A rede acha que esta imagem é do personagem: **{classes_invertido[classe_prevista].capitalize()}**")
print(f"📊 Probabilidades: {predicao}")

📷 Imagem: /content/homer2.png
🧠 A rede acha que esta imagem é do personagem: **Homer**
📊 Probabilidades: [[0.48822382 0.51177615]]


## ✅ Conclusão Geral do Trabalho

Este projeto teve como objetivo desenvolver um modelo de **classificação de imagens usando Redes Neurais Convolucionais (CNN)** com TensorFlow/Keras, para identificar os personagens **Bart Simpson** e **Homer Simpson** a partir de imagens.

---

### 🚀 Etapas Realizadas

1. **Organização e pré-processamento das imagens**  
   As imagens foram divididas automaticamente em treino (80%) e validação (20%) com `ImageDataGenerator`, que também normalizou e aumentou os dados com rotações, zooms, deslocamentos e espelhamentos.

2. **Criação da CNN**  
   A arquitetura foi composta por camadas convolucionais, pooling, batch normalization e dropout, promovendo um aprendizado robusto e ajudando a evitar overfitting.

3. **Treinamento do modelo**  
   O modelo foi treinado com **30 épocas**, observando a acurácia e a perda nos conjuntos de treino e validação a cada ciclo.

4. **Teste com imagens externas**  
   Imagens inéditas foram utilizadas ao final para avaliar a capacidade de generalização da rede.

---

### 📊 Resultados Obtidos

- Acurácia no treino: **acima de 90%**
- Acurácia na validação: **estável em torno de 68%**
- O modelo foi capaz de **classificar corretamente a maioria das imagens externas**, mesmo com estilos variados de desenho

---

### 🧠 Aprendizados

- Construção de CNNs com `Sequential` do Keras
- Uso de `ImageDataGenerator` para aumentar dados e separar validação
- Visualização e interpretação das probabilidades geradas pela rede
- Diagnóstico de possíveis overfittings e balanceamento de classes

---

### 🔧 Possíveis Melhorias Futuras

- Aumentar o dataset com mais imagens por classe
- Testar arquiteturas mais profundas (VGG16, MobileNet, etc.)
- Usar transfer learning com modelos pré-treinados
- Avaliar novas métricas (precision, recall, F1-score)

---

📂 Repositório desenvolvido para fins acadêmicos no curso de Inteligência Artificial.  
🔎 O projeto demonstra a eficácia de uma CNN simples na **classificação de imagens com poucas amostras**, usando boas práticas de pré-processamento e análise de desempenho.

